## 导入工具包

In [1]:
import time
import os
from tqdm import tqdm

import pandas as pd
import numpy as np

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt
%matplotlib inline

# 忽略烦人的红色提示
import warnings
warnings.filterwarnings("ignore")

# 获取计算硬件
# 有 GPU 就用 GPU，没有就用 CPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('device', device)

device cuda:0


## 图像预处理

In [2]:
from torchvision import transforms

# 训练集图像预处理：缩放裁剪、图像增强、转 Tensor、归一化
train_transform = transforms.Compose([transforms.RandomResizedCrop(224),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                     ])

# 测试集图像预处理-RCTN：缩放、裁剪、转 Tensor、归一化
test_transform = transforms.Compose([transforms.Resize(256),
                                     transforms.CenterCrop(224),
                                     transforms.ToTensor(),
                                     transforms.Normalize(
                                         mean=[0.485, 0.456, 0.406], 
                                         std=[0.229, 0.224, 0.225])
                                    ])

## 载入图像分类数据集

In [3]:
# 数据集文件夹路径
dataset_dir = '/media/tdx/bf69dd9d-303c-433a-870d-4348508b3b27/xcj/U/155/feizhongliu'
# dataset_dir = '/media/tdx/bf69dd9d-303c-433a-870d-4348508b3b27/xcj/U/155/patch_val'
# dataset_dir = '/media/tdx/bf69dd9d-303c-433a-870d-4348508b3b27/xcj/U/155/zhongliu'
# dataset_dir = '/media/tdx/bf69dd9d-303c-433a-870d-4348508b3b27/xcj/U/155ALL_SIZE_split'

In [4]:
# train_path = os.path.join(dataset_dir, 'train')
test_path = os.path.join(dataset_dir, 'val')
# print('训练集路径', train_path)
print('测试集路径', test_path)

from torchvision import datasets
# # 载入训练集
# train_dataset = datasets.ImageFolder(train_path, train_transform)
# 载入测试集
test_dataset = datasets.ImageFolder(test_path, test_transform)

# print('训练集图像数量', len(train_dataset))
# print('类别个数', len(train_dataset.classes))
# print('各类别名称', train_dataset.classes)
print('测试集图像数量', len(test_dataset))
print('类别个数', len(test_dataset.classes))
print('各类别名称', test_dataset.classes)

测试集路径 /media/tdx/bf69dd9d-303c-433a-870d-4348508b3b27/xcj/U/155/feizhongliu/val
测试集图像数量 139
类别个数 2
各类别名称 ['A_MSS', 'MSIH']


## 类别和索引号 映射字典

In [5]:
# 各类别名称
# class_names = train_dataset.classes
class_names = test_dataset.classes
n_class = len(class_names)
# 映射关系：类别 到 索引号
# train_dataset.class_to_idx
test_dataset.class_to_idx
# 映射关系：索引号 到 类别
idx_to_labels = {y:x for x,y in test_dataset.class_to_idx.items()}

In [6]:
idx_to_labels

{0: 'A_MSS', 1: 'MSIH'}

## 定义数据加载器DataLoader

In [7]:
def __getitem__(self, index):
    image = self.images[index]
    label = self.labels[index]
    name_id = self.names_ids[index]
    return image, label, name_id

In [8]:
from torch.utils.data import DataLoader

BATCH_SIZE = 1

# # 训练集的数据加载器
# train_loader = DataLoader(train_dataset,
#                           batch_size=BATCH_SIZE,
#                           shuffle=True,
#                           num_workers=4
#                          )

# 测试集的数据加载器
test_loader = DataLoader(test_dataset,
                         batch_size=BATCH_SIZE,
                         shuffle=False,
                         num_workers=4
                        )

## 导入训练需使用的工具包

In [9]:
from torchvision import models
import torch.optim as optim
from torch.optim import lr_scheduler

### 选择二：微调训练所有层

In [10]:
# model = models.resnet18(pretrained=True) # 载入预训练模型

# model.fc = nn.Linear(model.fc.in_features, n_class)

# optimizer = optim.Adam(model.parameters())

In [11]:
# model = torch.load('/media/tdx/bf69dd9d-303c-433a-870d-4348508b3b27/xcj/1/fianl/迁移学习图像分类/feizhongliu/vgg/checkpoints/best-0.777.pth'.format(best_test_accuracy))
# model = torch.load('/media/tdx/bf69dd9d-303c-433a-870d-4348508b3b27/xcj/1/fianl/迁移学习图像分类/feizhongliu/resnet18/checkpoints/best-0.835.pth')
# model = torch.load('/media/tdx/bf69dd9d-303c-433a-870d-4348508b3b27/xcj/1/fianl/迁移学习图像分类/feizhongliu/resnet18_specificity/checkpoints/best-0.129.pth')
# model = torch.load('/media/tdx/bf69dd9d-303c-433a-870d-4348508b3b27/xcj/1/fianl/迁移学习图像分类/feizhongliu/resnet18_specicicity_1000/checkpoints/best-0.516.pth')
model = torch.load('/media/tdx/bf69dd9d-303c-433a-870d-4348508b3b27/xcj/1/fianl/迁移学习图像分类/feizhongliu/resnet18_specicicity_1000/checkpoints/best-0.516.pth')


## 训练配置

In [12]:
model = model.to(device)

# # 交叉熵损失函数
criterion = nn.CrossEntropyLoss() 


In [13]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

from sklearn.metrics import confusion_matrix

import collections
# import torch.nn.functional as F
import scipy.special

## 函数：在整个测试集上评估

In [14]:
# def evaluate_testset():
#     '''
#     在整个测试集上评估，返回分类评估指标日志
#     '''

#     loss_list = []
#     labels_list = []
#     preds_list = []
#     preds_counter = collections.Counter()
#     labels_counter = collections.Counter()
    
#     with torch.no_grad():
#         for images, labels in test_loader: # 生成一个 batch 的数据和标注
# #             print("images",images)
#             images = images.to(device)
# #             print("images",images)
# #             print("labels",labels)
#             labels = labels.to(device)
# #             print("labels",labels)
            
#             outputs = model(images) # 输入模型，执行前向预测
# #             print("outputs",outputs)
#             # 获取整个测试集的标签类别和预测类别
#             _, preds = torch.max(outputs, 1) # 获得当前 batch 所有图像的预测类别
#             print("_,preds",_,preds)
#             print("preds",preds)
#             preds = preds.cpu().numpy()
#             print("preds",preds)
#             loss = criterion(outputs, labels) # 由 logit，计算当前 batch 中，每个样本的平均交叉熵损失函数值
#             loss = loss.detach().cpu().numpy()
#             outputs = outputs.detach().cpu().numpy()
# #             print("outputs",outputs)

            
            
# #             outputs_prob = F.softmax(outputs, dim=1)
            
#             outputs_prob = scipy.special.softmax(outputs, axis=1)
#             print("outputs_prob",outputs_prob)
            
#             outputs_prob = scipy.special.softmax(outputs, axis=1)
#             probabilitises_A = outputs_prob
#             print("probabilitises_A",probabilitises_A)


#             outputs = torch.from_numpy(outputs)
#             print("outputs",outputs)
# #             probabilitises = torch.sigmoid(outputs)
# #             probabilitises /= torch.sum(probabilitises)
# #             print("probabilitises",probabilitises)
#             probs_A = probabilitises_A.tolist()
#             probs_A1 = probs_A[0][0]
#             probs_A2 = probs_A[0][1]
#             print("probs_A1",probs_A1)
#             print("probs_A2",probs_A2)
#             print("\n")
            
            
#             if probs_A1 >= probs_A2:
#                 preds_list.append(0)
#             else:
#                 preds_list.append(1)
            
#             print("preds_list",preds_list)
            
            
#             labels = labels.detach().cpu().numpy()
#             print("labels",labels)
#             print("\n")

#             preds_counter.update(preds)
#             labels_counter.update(labels)
            
#             loss_list.append(loss)
#             labels_list.extend(labels)
# #             preds_list.extend(preds)
        
#     log_test = {}
#     log_test['epoch'] = epoch
#     print("labels_list",labels_list)
#     print("preds_list",preds_list)
    
#     print("labels_counter",labels_counter)
#     print("preds_counter",preds_counter)
    
#     # 计算分类评估指标
#     log_test['test_loss'] = np.mean(loss)
#     log_test['test_accuracy'] = accuracy_score(labels_list, preds_list)
# #     log_test['test_precision'] = precision_score(labels_list, preds_list, average='macro')
# #     log_test['test_recall'] = recall_score(labels_list, preds_list, average='macro')
# #     log_test['test_f1-score'] = f1_score(labels_list, preds_list, average='macro')

#     log_test['test_precision'] = precision_score(labels_list, preds_list)
#     log_test['test_recall'] = recall_score(labels_list, preds_list)
#     log_test['test_f1-score'] = f1_score(labels_list, preds_list)
        
#     tn,fp,fn,tp = confusion_matrix(labels_list,preds_list).ravel()
#     log_test['test_sensitivity'] = tp / (tp + fn)
#     log_test['test_specificity'] = tn / (tn + fp)
#     log_test['test_PPV'] = tp / (tp + fp)
#     log_test['test_NPV'] = tn / (tn + fn)
    
#     return log_test

In [77]:
import collections
import torch
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

def evaluate_testset():
    '''
    在整个测试集上评估，返回分类评估指标日志
    '''
    Prefix_list = []
    loss_list = []
    labels_list = []
    preds_list = []
    names_ids_list = []  # 存储图片的名称或ID
    preds_counter = collections.Counter()
    labels_counter = collections.Counter()

    with torch.no_grad():
        for i, (images, labels) in enumerate(test_loader):  # 生成一个 batch 的数据和标签
            images = images.to(device)
            labels = labels.to(device)

            # 获取模型预测结果
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            preds = preds.cpu().numpy()
            labels = labels.cpu().numpy()

            preds_counter.update(preds)
            labels_counter.update(labels)
            labels_list.extend(labels)

            # 输出标签、预测结果和图片名称或ID
            for j in range(len(labels)):
                name_id = test_loader.dataset.samples[i * test_loader.batch_size + j][0]
                name_id = os.path.basename(name_id)  # 提取文件名作为名称或ID
                names_ids_list.append(name_id)

                
                
                     # 创建一个字典用于存储具有相同前7位的name_id的列表
                name_ids_dict = {}

                # 遍历name_ids_list
                for name_id in names_ids_list:
                    prefix = name_id[:7]  # 获取前7位
                    if prefix in name_ids_dict:
                        name_ids_dict[prefix].append(name_id)  # 如果前7位已经存在于字典中，将name_id添加到对应的列表中
                    else:
                        name_ids_dict[prefix] = [name_id]  # 如果前7位不存在于字典中，创建一个新的列表，并将name_id加入其中

                         # 输出具有相同前7位的name_id的Label和Prediction
                for prefix, ids in name_ids_dict.items():
                    print("prefix: {}".format(prefix))
                    Prefix_list.append(prefix)
                    for name_id in ids:
                        print("Label: {}, Prediction: {}, Name/ID: {}".format(labels[j], preds[j], names_ids_list[-1]))
#                         print("Name/ID: {}, Label: {}, Prediction: {}".format(name_id, labels[name_id], predictions[name_id]))
                        print()
  
                
                
#                 if name_id == "s122722_c5.tif": 
#                     print("Label: {}, Prediction: {}, Name/ID: {}".format(labels[j], preds[j], names_ids_list[-1]))
                
        print("labels_list:",labels_list)
        print("names_ids_list:",names_ids_list)
        print("Prefix_list:",Prefix_list)
        
        
       
        
        
        
        
    log_test = {}
    log_test['epoch'] = epoch

   

#     # 计算分类评估指标
#     log_test['test_accuracy'] = accuracy_score(labels_list, preds_list)
#     log_test['test_precision'] = precision_score(labels_list, preds_list)
#     log_test['test_recall'] = recall_score(labels_list, preds_list)
#     log_test['test_f1-score'] = f1_score(labels_list, preds_list)

#     tn, fp, fn, tp = confusion_matrix(labels_list, preds_list).ravel()
#     log_test['test_sensitivity'] = tp / (tp + fn)
#     log_test['test_specificity'] = tn / (tn + fp)
#     log_test['test_PPV'] = tp / (tp + fp)
#     log_test['test_NPV'] = tn / (tn + fn)

    return log_test

In [78]:
#记录日志
epoch = 0
batch_idx = 0
best_test_accuracy = 0

In [79]:
# 日志-测试集
df_test_log = pd.DataFrame()
log_test = {}
log_test['epoch'] = 0
log_test.update(evaluate_testset())
df_test_log = df_test_log.append(log_test, ignore_index=True)

prefix: s093023
Label: 0, Prediction: 1, Name/ID: s093023_m2.tif

prefix: s093023
Label: 0, Prediction: 1, Name/ID: s093023_m3.tif

Label: 0, Prediction: 1, Name/ID: s093023_m3.tif

prefix: s093023
Label: 0, Prediction: 1, Name/ID: s093023_s2.tif

Label: 0, Prediction: 1, Name/ID: s093023_s2.tif

Label: 0, Prediction: 1, Name/ID: s093023_s2.tif

prefix: s093023
Label: 0, Prediction: 1, Name/ID: s093023_s4.tif

Label: 0, Prediction: 1, Name/ID: s093023_s4.tif

Label: 0, Prediction: 1, Name/ID: s093023_s4.tif

Label: 0, Prediction: 1, Name/ID: s093023_s4.tif

prefix: s093023
Label: 0, Prediction: 1, Name/ID: s093453_s5.tif

Label: 0, Prediction: 1, Name/ID: s093453_s5.tif

Label: 0, Prediction: 1, Name/ID: s093453_s5.tif

Label: 0, Prediction: 1, Name/ID: s093453_s5.tif

prefix: s093453
Label: 0, Prediction: 1, Name/ID: s093453_s5.tif

prefix: s093023
Label: 0, Prediction: 1, Name/ID: s093566_c1.tif

Label: 0, Prediction: 1, Name/ID: s093566_c1.tif

Label: 0, Prediction: 1, Name/ID: s093

prefix: s093023
Label: 0, Prediction: 1, Name/ID: s102447_s3.tif

Label: 0, Prediction: 1, Name/ID: s102447_s3.tif

Label: 0, Prediction: 1, Name/ID: s102447_s3.tif

Label: 0, Prediction: 1, Name/ID: s102447_s3.tif

prefix: s093453
Label: 0, Prediction: 1, Name/ID: s102447_s3.tif

prefix: s093566
Label: 0, Prediction: 1, Name/ID: s102447_s3.tif

Label: 0, Prediction: 1, Name/ID: s102447_s3.tif

prefix: s093710
Label: 0, Prediction: 1, Name/ID: s102447_s3.tif

Label: 0, Prediction: 1, Name/ID: s102447_s3.tif

prefix: s093794
Label: 0, Prediction: 1, Name/ID: s102447_s3.tif

Label: 0, Prediction: 1, Name/ID: s102447_s3.tif

Label: 0, Prediction: 1, Name/ID: s102447_s3.tif

prefix: s093798
Label: 0, Prediction: 1, Name/ID: s102447_s3.tif

Label: 0, Prediction: 1, Name/ID: s102447_s3.tif

prefix: s093799
Label: 0, Prediction: 1, Name/ID: s102447_s3.tif

Label: 0, Prediction: 1, Name/ID: s102447_s3.tif

prefix: s093936
Label: 0, Prediction: 1, Name/ID: s102447_s3.tif

prefix: s094227
Label:

prefix: s093023
Label: 0, Prediction: 1, Name/ID: s103550_s5.tif

Label: 0, Prediction: 1, Name/ID: s103550_s5.tif

Label: 0, Prediction: 1, Name/ID: s103550_s5.tif

Label: 0, Prediction: 1, Name/ID: s103550_s5.tif

prefix: s093453
Label: 0, Prediction: 1, Name/ID: s103550_s5.tif

prefix: s093566
Label: 0, Prediction: 1, Name/ID: s103550_s5.tif

Label: 0, Prediction: 1, Name/ID: s103550_s5.tif

prefix: s093710
Label: 0, Prediction: 1, Name/ID: s103550_s5.tif

Label: 0, Prediction: 1, Name/ID: s103550_s5.tif

prefix: s093794
Label: 0, Prediction: 1, Name/ID: s103550_s5.tif

Label: 0, Prediction: 1, Name/ID: s103550_s5.tif

Label: 0, Prediction: 1, Name/ID: s103550_s5.tif

prefix: s093798
Label: 0, Prediction: 1, Name/ID: s103550_s5.tif

Label: 0, Prediction: 1, Name/ID: s103550_s5.tif

prefix: s093799
Label: 0, Prediction: 1, Name/ID: s103550_s5.tif

Label: 0, Prediction: 1, Name/ID: s103550_s5.tif

prefix: s093936
Label: 0, Prediction: 1, Name/ID: s103550_s5.tif

prefix: s094227
Label:

prefix: s093023
Label: 0, Prediction: 1, Name/ID: s104539_m3.tif

Label: 0, Prediction: 1, Name/ID: s104539_m3.tif

Label: 0, Prediction: 1, Name/ID: s104539_m3.tif

Label: 0, Prediction: 1, Name/ID: s104539_m3.tif

prefix: s093453
Label: 0, Prediction: 1, Name/ID: s104539_m3.tif

prefix: s093566
Label: 0, Prediction: 1, Name/ID: s104539_m3.tif

Label: 0, Prediction: 1, Name/ID: s104539_m3.tif

prefix: s093710
Label: 0, Prediction: 1, Name/ID: s104539_m3.tif

Label: 0, Prediction: 1, Name/ID: s104539_m3.tif

prefix: s093794
Label: 0, Prediction: 1, Name/ID: s104539_m3.tif

Label: 0, Prediction: 1, Name/ID: s104539_m3.tif

Label: 0, Prediction: 1, Name/ID: s104539_m3.tif

prefix: s093798
Label: 0, Prediction: 1, Name/ID: s104539_m3.tif

Label: 0, Prediction: 1, Name/ID: s104539_m3.tif

prefix: s093799
Label: 0, Prediction: 1, Name/ID: s104539_m3.tif

Label: 0, Prediction: 1, Name/ID: s104539_m3.tif

prefix: s093936
Label: 0, Prediction: 1, Name/ID: s104539_m3.tif

prefix: s094227
Label:

prefix: s093023
Label: 0, Prediction: 1, Name/ID: s110769_c3.tif

Label: 0, Prediction: 1, Name/ID: s110769_c3.tif

Label: 0, Prediction: 1, Name/ID: s110769_c3.tif

Label: 0, Prediction: 1, Name/ID: s110769_c3.tif

prefix: s093453
Label: 0, Prediction: 1, Name/ID: s110769_c3.tif

prefix: s093566
Label: 0, Prediction: 1, Name/ID: s110769_c3.tif

Label: 0, Prediction: 1, Name/ID: s110769_c3.tif

prefix: s093710
Label: 0, Prediction: 1, Name/ID: s110769_c3.tif

Label: 0, Prediction: 1, Name/ID: s110769_c3.tif

prefix: s093794
Label: 0, Prediction: 1, Name/ID: s110769_c3.tif

Label: 0, Prediction: 1, Name/ID: s110769_c3.tif

Label: 0, Prediction: 1, Name/ID: s110769_c3.tif

prefix: s093798
Label: 0, Prediction: 1, Name/ID: s110769_c3.tif

Label: 0, Prediction: 1, Name/ID: s110769_c3.tif

prefix: s093799
Label: 0, Prediction: 1, Name/ID: s110769_c3.tif

Label: 0, Prediction: 1, Name/ID: s110769_c3.tif

prefix: s093936
Label: 0, Prediction: 1, Name/ID: s110769_c3.tif

prefix: s094227
Label:

...

prefix: s093023
Label: 0, Prediction: 1, Name/ID: s116228_c5.tif

Label: 0, Prediction: 1, Name/ID: s116228_c5.tif

Label: 0, Prediction: 1, Name/ID: s116228_c5.tif

Label: 0, Prediction: 1, Name/ID: s116228_c5.tif

prefix: s093453
Label: 0, Prediction: 1, Name/ID: s116228_c5.tif

prefix: s093566
Label: 0, Prediction: 1, Name/ID: s116228_c5.tif

Label: 0, Prediction: 1, Name/ID: s116228_c5.tif

prefix: s093710
Label: 0, Prediction: 1, Name/ID: s116228_c5.tif

Label: 0, Prediction: 1, Name/ID: s116228_c5.tif

prefix: s093794
Label: 0, Prediction: 1, Name/ID: s116228_c5.tif

Label: 0, Prediction: 1, Name/ID: s116228_c5.tif

Label: 0, Prediction: 1, Name/ID: s116228_c5.tif

prefix: s093798
Label: 0, Prediction: 1, Name/ID: s116228_c5.tif

Label: 0, Prediction: 1, Name/ID: s116228_c5.tif

prefix: s093799
Label: 0, Prediction: 1, Name/ID: s116228_c5.tif

Label: 0, Prediction: 1, Name/ID: s116228_c5.tif

prefix: s093936
Label: 0, Prediction: 1, Name/ID: s116228_c5.tif

prefix: s094227
Label:

prefix: s093023
Label: 0, Prediction: 1, Name/ID: s123076_s2.tif

Label: 0, Prediction: 1, Name/ID: s123076_s2.tif

Label: 0, Prediction: 1, Name/ID: s123076_s2.tif

Label: 0, Prediction: 1, Name/ID: s123076_s2.tif

prefix: s093453
Label: 0, Prediction: 1, Name/ID: s123076_s2.tif

prefix: s093566
Label: 0, Prediction: 1, Name/ID: s123076_s2.tif

Label: 0, Prediction: 1, Name/ID: s123076_s2.tif

prefix: s093710
Label: 0, Prediction: 1, Name/ID: s123076_s2.tif

Label: 0, Prediction: 1, Name/ID: s123076_s2.tif

prefix: s093794
Label: 0, Prediction: 1, Name/ID: s123076_s2.tif

Label: 0, Prediction: 1, Name/ID: s123076_s2.tif

Label: 0, Prediction: 1, Name/ID: s123076_s2.tif

prefix: s093798
Label: 0, Prediction: 1, Name/ID: s123076_s2.tif

Label: 0, Prediction: 1, Name/ID: s123076_s2.tif

prefix: s093799
Label: 0, Prediction: 1, Name/ID: s123076_s2.tif

Label: 0, Prediction: 1, Name/ID: s123076_s2.tif

prefix: s093936
Label: 0, Prediction: 1, Name/ID: s123076_s2.tif

prefix: s094227
Label:

prefix: s093023
Label: 1, Prediction: 1, Name/ID: s110360_c1.tif

Label: 1, Prediction: 1, Name/ID: s110360_c1.tif

Label: 1, Prediction: 1, Name/ID: s110360_c1.tif

Label: 1, Prediction: 1, Name/ID: s110360_c1.tif

prefix: s093453
Label: 1, Prediction: 1, Name/ID: s110360_c1.tif

prefix: s093566
Label: 1, Prediction: 1, Name/ID: s110360_c1.tif

Label: 1, Prediction: 1, Name/ID: s110360_c1.tif

prefix: s093710
Label: 1, Prediction: 1, Name/ID: s110360_c1.tif

Label: 1, Prediction: 1, Name/ID: s110360_c1.tif

prefix: s093794
Label: 1, Prediction: 1, Name/ID: s110360_c1.tif

Label: 1, Prediction: 1, Name/ID: s110360_c1.tif

Label: 1, Prediction: 1, Name/ID: s110360_c1.tif

prefix: s093798
Label: 1, Prediction: 1, Name/ID: s110360_c1.tif

Label: 1, Prediction: 1, Name/ID: s110360_c1.tif

prefix: s093799
Label: 1, Prediction: 1, Name/ID: s110360_c1.tif

Label: 1, Prediction: 1, Name/ID: s110360_c1.tif

prefix: s093936
Label: 1, Prediction: 1, Name/ID: s110360_c1.tif

prefix: s094227
Label:

prefix: s093023
Label: 1, Prediction: 0, Name/ID: s112409_c1.tif

Label: 1, Prediction: 0, Name/ID: s112409_c1.tif

Label: 1, Prediction: 0, Name/ID: s112409_c1.tif

Label: 1, Prediction: 0, Name/ID: s112409_c1.tif

prefix: s093453
Label: 1, Prediction: 0, Name/ID: s112409_c1.tif

prefix: s093566
Label: 1, Prediction: 0, Name/ID: s112409_c1.tif

Label: 1, Prediction: 0, Name/ID: s112409_c1.tif

prefix: s093710
Label: 1, Prediction: 0, Name/ID: s112409_c1.tif

Label: 1, Prediction: 0, Name/ID: s112409_c1.tif

prefix: s093794
Label: 1, Prediction: 0, Name/ID: s112409_c1.tif

Label: 1, Prediction: 0, Name/ID: s112409_c1.tif

Label: 1, Prediction: 0, Name/ID: s112409_c1.tif

prefix: s093798
Label: 1, Prediction: 0, Name/ID: s112409_c1.tif

Label: 1, Prediction: 0, Name/ID: s112409_c1.tif

prefix: s093799
Label: 1, Prediction: 0, Name/ID: s112409_c1.tif

Label: 1, Prediction: 0, Name/ID: s112409_c1.tif

prefix: s093936
Label: 1, Prediction: 0, Name/ID: s112409_c1.tif

prefix: s094227
Label:

prefix: s093023
Label: 1, Prediction: 1, Name/ID: s123209_m3.tif

Label: 1, Prediction: 1, Name/ID: s123209_m3.tif

Label: 1, Prediction: 1, Name/ID: s123209_m3.tif

Label: 1, Prediction: 1, Name/ID: s123209_m3.tif

prefix: s093453
Label: 1, Prediction: 1, Name/ID: s123209_m3.tif

prefix: s093566
Label: 1, Prediction: 1, Name/ID: s123209_m3.tif

Label: 1, Prediction: 1, Name/ID: s123209_m3.tif

prefix: s093710
Label: 1, Prediction: 1, Name/ID: s123209_m3.tif

Label: 1, Prediction: 1, Name/ID: s123209_m3.tif

prefix: s093794
Label: 1, Prediction: 1, Name/ID: s123209_m3.tif

Label: 1, Prediction: 1, Name/ID: s123209_m3.tif

Label: 1, Prediction: 1, Name/ID: s123209_m3.tif

prefix: s093798
Label: 1, Prediction: 1, Name/ID: s123209_m3.tif

Label: 1, Prediction: 1, Name/ID: s123209_m3.tif

prefix: s093799
Label: 1, Prediction: 1, Name/ID: s123209_m3.tif

Label: 1, Prediction: 1, Name/ID: s123209_m3.tif

prefix: s093936
Label: 1, Prediction: 1, Name/ID: s123209_m3.tif

prefix: s094227
Label:

## 在测试集上评价

In [ ]:
model.eval()
print(evaluate_testset())